# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = "TRAAAVO128F93133D4.json"

In [5]:
filepath = "data/song_data/"

In [22]:
js = get_files(filepath)#pd.read_json("data/song_data/A/A/A/TRAAAVO128F93133D4.json",lines=True,orient='columns')
js[0]
df = pd.read_json(js[0],lines=True,orient='columns')
df.head(50)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
5,LMFAO,Logged In,Jacob,M,4,Klein,227.99628,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,I'm In Miami Bitch,200,1543541072796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
6,DJ Dizzy,Logged In,Jacob,M,5,Klein,221.15220,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Sexy Bitch,200,1543541299796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
7,Fish Go Deep & Tracey K,Logged In,Jacob,M,6,Klein,377.41669,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,The Cure & The Cause (Dennis Ferrer Remix),200,1543541520796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
8,None,Logged In,Alivia,F,0,Terrell,NaN,free,"Parkersburg-Vienna, WV",GET,Home,1.540505e+12,1070,None,200,1543541644796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",4
9,M83,Logged In,Jacob,M,7,Klein,96.18240,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Staring At Me,200,1543541897796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[['song_id','title','artist_id','year','duration']].values[0].tolist()
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
song_table_insert = " INSERT INTO songs (song_id, title, artist_id, year, duration ) VALUES (%s, %s, %s, %s, %s)"
cur.execute(song_table_insert, song_data)#song_data['song_id'],song_data['title'],song_data['artist_id'],song_data['year'],song_data['duration'])
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values[0].tolist()
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
artist_table_insert = " INSERT INTO artists (artist_id, name, location, latitude, longitude ) VALUES (%s, %s, %s, %s, %s)"
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
#log_files = 

In [12]:
filepath = "data/log_data"

In [13]:
js2 = get_files(filepath)#pd.read_json("data/song_data/A/A/A/TRAAAVO128F93133D4.json",lines=True,orient='columns')
js2[0]
df2 = pd.read_json(js2[0],lines=True,orient='columns')
df2.head(50)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
5,LMFAO,Logged In,Jacob,M,4,Klein,227.99628,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,I'm In Miami Bitch,200,1543541072796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
6,DJ Dizzy,Logged In,Jacob,M,5,Klein,221.15220,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Sexy Bitch,200,1543541299796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
7,Fish Go Deep & Tracey K,Logged In,Jacob,M,6,Klein,377.41669,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,The Cure & The Cause (Dennis Ferrer Remix),200,1543541520796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
8,None,Logged In,Alivia,F,0,Terrell,NaN,free,"Parkersburg-Vienna, WV",GET,Home,1.540505e+12,1070,None,200,1543541644796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",4
9,M83,Logged In,Jacob,M,7,Klein,96.18240,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Staring At Me,200,1543541897796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df2 = df2[df2['page'] == 'NextSong']
df2 = df2.head(1)
df2

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91


In [28]:
from datetime import datetime
ts = df2['ts']/1000.0
ts_convert = pd.to_datetime(ts)
df2['date_time'] = pd.to_datetime(ts_convert)

column_labels = ('time','hour','day','week','month','year','weekday')
time_df = pd.DataFrame(columns= column_labels)

time_df['time'] = df2.date_time.dt.time
time_df['hour'] = df2.date_time.dt.hour
time_df['day'] = df2.date_time.dt.day
time_df['week'] = df2.date_time.dt.week
time_df['month'] = df2.date_time.dt.month
time_df['year'] = df2.date_time.dt.year
time_df['weekday'] = df2.date_time.dt.weekday

time_df = time_df.head(1)
time_df

,time,hour,day,week,month,year,weekday
0,00:00:01.543537,0,1,1,1,1970,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [16]:
time_table_insert = " INSERT INTO time (start_time, hour, day, week, month, year, day_week ) VALUES (%s, %s, %s, %s, %s, %s, %s)"
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()
    

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [17]:
user_df = df2[['userId', 'firstName','lastName', 'gender', 'level']]
user_df = user_df.head(1)
user_df

,userId,firstName,lastName,gender,level
0,91,Jayden,Bell,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [18]:
user_table_insert = " INSERT INTO users (user_id, first_name, last_name, gender, level) VALUES (%s, %s, %s, %s, %s) ON CONFLICT (user_id) DO UPDATE SET level = EXCLUDED.level"
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [19]:
song_select = " SELECT B.artist_id, A.song_id  FROM SONGS A LEFT JOIN  ARTISTS B ON A.artist_id = B.artist_id  where a.title = %s and b.name =%s and a.duration = %s "

songplay_table_insert = " INSERT into songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
for index, row in df2.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    
    print(songid)
    print(artistid)
    print(row)

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent )
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()
    

None
None
artist                                               Stephen Lynch
auth                                                     Logged In
firstName                                                   Jayden
gender                                                           M
itemInSession                                                    0
lastName                                                      Bell
length                                                     182.857
level                                                         free
location                           Dallas-Fort Worth-Arlington, TX
method                                                         PUT
page                                                      NextSong
registration                                           1.54099e+12
sessionId                                                      829
song                                             Jim Henson's Dead
status                                              

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [20]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.